Just a simple script to show the image and label data for each patient as well as a few relevant clinical notes

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from skimage.util.montage import montage2d
from skimage.color import label2rgb
import os
import h5py
import seaborn as sns
sns.set_style("whitegrid", {'axes.grid': False})

# Data Overview
The the CT data (as a radiograph), the PET data (as an MIP image) and the label (as an MIP image) for the a series of patients. Verify that the images and labelings match up with text descriptions

In [ ]:
%matplotlib inline
with h5py.File(os.path.join('..', 'input', 'lab_petct_vox_5.00mm.h5'), 'r') as p_data:
    fig, (c_ax1, c_ax2, c_ax3) = plt.subplots(1, 3, figsize=(3, 3), dpi = 250)
    for (p_id, ct_img), pt_img, lab_img in zip(
                                   p_data['ct_data'].items(),
                                   p_data['pet_data'].values(),
                                   p_data['label_data'].values()
                                                           ):
        c_ax1.imshow(np.sum(ct_img,1).squeeze()[::-1,:], cmap = 'bone')
        c_ax1.set_title('CT')
        c_ax1.axis('off')
        
        c_ax2.imshow(np.sqrt(np.max(pt_img,1).squeeze()[::-1,:]), cmap = 'magma')
        c_ax2.set_title('PET\n(sqrt)')
        c_ax2.axis('off')
        
        c_ax3.imshow(np.max(lab_img,1).squeeze()[::-1,:], cmap = 'gist_earth')
        c_ax3.set_title('Label')
        c_ax3.axis('off')
        cur_ct_img = np.sum(ct_img,1).squeeze()[::-1,:]
        cur_pet_img = np.sum(pt_img,1).squeeze()[::-1,:]
        cur_lab_img = np.max(lab_img,1).squeeze()[::-1,:]
        break # only load the first patient

# From Slices to Tiles
Here we go from the simple images to many little tiles the **i**, **j** indices iterate over the image while the $i_k$ and $j_k$ iterate over overlapping regions

In [ ]:
ct_tiles = []
pet_tiles = []
lab_tiles = []
from tqdm import tqdm
for i in tqdm(range(0, cur_ct_img.shape[0], 16)):
    for j in range(0, cur_ct_img.shape[1], 16):
        for i_k in range(0, 16, 3):
            for j_k in range(0, 16, 3):
                ct_tiles += [cur_ct_img[i+i_k:(i+i_k+16),j+j_k:(j+j_k+16)]]
                pet_tiles += [cur_pet_img[i+i_k:(i+i_k+16),j+j_k:(j+j_k+16)]]
                lab_tiles += [cur_lab_img[i+i_k:(i+i_k+16),j+j_k:(j+j_k+16)]]

In [ ]:
# we want to remove all tiles which do not have the right size (borders are a problem)
n_ct_tiles = [c_tile for c_tile in ct_tiles if c_tile.shape == (16,16)]
n_pet_tiles = [c_tile for c_tile in pet_tiles if c_tile.shape == (16,16)]
n_lab_tiles = [c_tile for c_tile in lab_tiles if c_tile.shape == (16,16)]
fig, (ax1, ax2, ax3) = plt.subplots(1,3)
ax1.imshow(montage2d(np.stack(n_ct_tiles,0)), cmap = 'bone')
ax2.imshow(montage2d(np.stack(n_pet_tiles,0)), cmap = 'bone')
ax3.imshow(montage2d(np.stack(n_lab_tiles,0)), cmap = 'gist_earth')

In [ ]:
# Calculate the score for each tile and a binary class lab_class which can be used
# later for stratificaton
lab_score = [np.mean(c_tile) for c_tile in n_lab_tiles]
lab_class = [c_score > 0 for c_score in lab_score]

# Create a flat list of all the tiles
ct_tile_flat = [c_tile.flatten() for c_tile in n_ct_tiles]
pet_tile_flat = [p_tile.flatten() for p_tile in n_pet_tiles]


In [ ]:
from sklearn.model_selection import train_test_split
train_tiles, test_tile, train_score, test_score = train_test_split(np.stack(ct_tile_flat), lab_score, 
                                                                   train_size = 0.8, stratify = lab_class, random_state = 1234)
print('Training size', train_tiles.shape, 'Testing size', test_tile.shape)
print('Train tumor tiles', np.where(train_score), 'Test tumor tiles', np.where(test_score))

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knn1_model = KNeighborsRegressor(1, algorithm = 'brute')

In [ ]:
knn1_model.fit(train_tiles, train_score)

In [ ]:
test_predictions = knn1_model.predict(test_tile)
print('Predicted Results', test_predictions)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(np.array(test_score)>0, np.array(test_predictions)>0)

In [ ]:
fig, ax1 = plt.subplots(1,1, figsize=(8, 5))
ax1.plot([0,1], [0,1], 'b-', label = 'Ideal Model')
ax1.plot(test_score, test_predictions, 'ro', label = 'Current Results')
ax1.set_xlabel('Ground Truth Tumor Score')
ax1.legend()
ax1.set_ylabel('Predicted Tumor Score')
ax1.set_xlim(-0.1,1.1)

In [ ]:
# Make a more complicated model with Neighbors = 2

In [ ]:
knn2_model = KNeighborsRegressor(4)
knn2_model.fit(train_tiles, train_score)
test2_predictions = knn2_model.predict(test_tile)
print('Predicted Results', test2_predictions)
confusion_matrix(np.array(test_score)>0, np.array(test2_predictions)>0)

In [ ]:
fig, ax1 = plt.subplots(1,1, figsize=(8, 5))
ax1.plot([0,1], [0,1], 'b-', label = 'Ideal Model')
ax1.plot(test_score, test_predictions, 'ro', label = 'KNN1 Results')
ax1.plot(test_score, test2_predictions, 'go', label = 'KNN2 Results')
ax1.set_xlabel('Ground Truth Tumor Score')
ax1.legend()
ax1.set_ylabel('Predicted Tumor Score')
ax1.set_xlim(-0.1,1.1)

# Add PET data to tiles
Instead of having just CT tiles, we now make PETCT tiles to train with

In [ ]:
petct_tile_flat = np.stack([np.hstack([c_ct_tile.flatten(), c_pet_tile.flatten()])  for c_ct_tile, c_pet_tile in zip(n_ct_tiles, n_pet_tiles)])
print('PETCT Tile Shape',petct_tile_flat.shape)

In [ ]:
petct_train_tiles, petct_test_tile, petct_train_score, petct_test_score = train_test_split(petct_tile_flat, lab_score, 
                                                                   train_size = 0.8, stratify = lab_class, random_state = 1234)
print('Training size', petct_train_tiles.shape, 'Testing size', petct_test_tile.shape)
print('Train tumor tiles', len(np.where(petct_train_score)[0]), 
      ', Test tumor tiles', len(np.where(petct_test_score)[0]))

In [ ]:
petct_knn1_model = KNeighborsRegressor(1, algorithm = 'brute')
petct_knn1_model.fit(petct_train_tiles, petct_train_score)

In [ ]:
petct_test_predictions = petct_knn1_model.predict(petct_test_tile)
print('Predicted Results', np.mean(petct_test_predictions))

In [ ]:
fig, ax1 = plt.subplots(1,1, figsize=(8, 5))
ax1.plot([0,1], [0,1], 'b-', label = 'Ideal Model')
ax1.plot(test_score, test_predictions, 'ro', label = 'KNN1 Results')
ax1.plot(test_score, test2_predictions, 'go', label = 'KNN2 Results')
ax1.plot(test_score, petct_test_predictions, 'yD', label = 'PETCT KNN1 Results')
ax1.set_xlabel('Ground Truth Tumor Score')
ax1.legend()
ax1.set_ylabel('Predicted Tumor Score')
ax1.set_xlim(-0.1,1.1)

# PET Data
It seems the PET does not show any effect on the training. Why is a question that needs to be answered

Try PET only training

In [ ]:
pet_knn1_model = KNeighborsRegressor(1, algorithm = 'brute')
pet_train_tiles, pet_test_tile, pet_train_score, pet_test_score = train_test_split(np.stack(pet_tile_flat), lab_score, 
                                                                   train_size = 0.8, stratify = lab_class, random_state = 1234)
print('Training size', pet_train_tiles.shape, 'Testing size', pet_test_tile.shape)
print('Train tumor tiles', np.where(pet_train_score), 'Test tumor tiles', np.where(pet_test_score))

In [ ]:
# train
pet_knn1_model.fit(pet_train_tiles, pet_train_score)
pet_test_predictions = pet_knn1_model.predict(pet_test_tile)

In [ ]:
# show confusion matrix
confusion_matrix(np.array(pet_test_score)>0, np.array(pet_test_predictions)>0)

In [ ]:
fig, ax1 = plt.subplots(1,1, figsize=(8, 5))
ax1.plot([0,1], [0,1], 'b-', label = 'Ideal Model')
ax1.plot(test_score, test2_predictions, 'go', label = 'KNN2 Results')
ax1.plot(test_score, petct_test_predictions, 'yD', label = 'PETCT KNN1 Results')
ax1.plot(test_score, pet_test_predictions, 'bo', label = 'KNN1 Results (PET only)')
ax1.set_xlabel('Ground Truth Tumor Score')
ax1.legend()
ax1.set_ylabel('Predicted Tumor Score')
ax1.set_xlim(-0.1, 1.1)
ax1.set_ylim(-0.1, 1.1)

In [ ]:
pet_knn2_model = KNeighborsRegressor(4)

# train
pet_knn2_model.fit(pet_train_tiles, pet_train_score)
pet_test_knn2_predictions = pet_knn2_model.predict(pet_test_tile)

# show confusion matrix
confusion_matrix(np.array(pet_test_score)>0, np.array(pet_test_knn2_predictions)>0)

In [ ]:
fig, ax1 = plt.subplots(1,1)
ax1.plot([0,1], [0,1], 'b-', label = 'Ideal Model')
ax1.plot(test_score, test_predictions, 'ro', label = 'KNN1 Results')
ax1.plot(test_score, test2_predictions, 'go', label = 'KNN2 Results')
ax1.plot(test_score, petct_test_predictions, 'y.', label = 'PETCT KNN1 Results')
ax1.plot(test_score, pet_test_predictions, 'bo', label = 'KNN1 Results (PET only)')
ax1.plot(test_score, pet_test_knn2_predictions, 'sy', label = 'KNN2 Results (PET only)')
ax1.set_xlabel('Ground Truth Tumor Score')
ax1.legend()
ax1.set_ylabel('Predicted Tumor Score')
ax1.set_xlim(-0.1, 1.1)
ax1.set_ylim(-0.1, 1.1)